# HEALPix Python Project

Author: Yannick Hénin
University of Strasbourg

First we import our python module

In [146]:
from moc_library import Loader, Moc_tree

We define useful constant in this cell such as:
- name of the catalogue to query
- max number of elements in the table
- order of the initial healpix cell

__Note :__ Below are some usable catalogues:

    - 'II/7A/catalog' : UBVRIJKLMNH Photoelectric Catalogue (Morel+ 1978)
    - 'J/ApJ/831/67/table1' : Galactic CHaMP. III. 12CO dense clump properties (Barnes+, 2016)
    - 'J/ApJ/804/L15/data' : SDSS-DR7 broad-line QSOs (Sun+, 2015)
    - 'VII/26D/catalog' : Uppsala General Catalogue of Galaxies (UGC) (1973)
    - 'I/298/table3' :  Catalog of Northern stars with annual proper motions larger than 0.15" (2005)
    - 'V/148/morx'
    - 'J/ApJS/236/30/hatlas2ngp'

In [147]:
CATALOGUE = 'II/7A/catalog'
OUT_MAX = "unlimited" # One can specify the limit number or 'unlmited'
ORDER = 4

The library is built using the astropy library, it is hence able to query the table from Vizier easily. 
This step is made by the Loader class which has only two methods:
- get_votable() : Allow one to get the votable
- build_healpix_table() : Construct a healpix column from (ra,dec) and add it to the table

Let's first query the catalogue and have a look at the table queried.
__Note :__ The query can be made from CDS (Strasbourg) Vizier or CFA (Harvard) Vizier, this is specified in the source argument 

In [148]:
loader = Loader(_catalogue=CATALOGUE, _out_max=OUT_MAX, _order=ORDER, source='CFA')
votable = loader.get_votable()
print(votable)

Querying: 'II/7A/catalog' from Vizier
Coordinate system of the table: eq_FK5

recno JP11     LID     n_LID   U    ...  N    H    ref     _RA       _DE   
                              mag   ... mag  mag           deg       deg   
----- ---- ----------- ----- ------ ... --- ------ ---- --------- ---------
    1    1  0.00000444           -- ...  --  6.640 8130  40.17863   1.19868
    2    1  0.00000444           -- ...  --  6.640 8130  40.17863   1.19868
    3    3  0.00004030       15.340 ...  --  4.100 8057 282.60583   0.79167
    4    4  0.00004064       13.570 ...  --  2.390 8057 284.38983   0.45552
    5    5  0.00101522           -- ...  --  6.280 8140 102.08778   1.21897
    6    5  0.00101522           -- ...  --  6.320 8140 102.08778   1.21897
    7    7  0.00102447       12.340 ...  --     -- 8008 157.23146   0.84100
    8    7  0.00102447       12.370 ...  --  5.640 8075 157.23146   0.84100
    9    7  0.00102447           -- ...  --  5.600 8130 157.23146   0.84100
   10    7

Let's now add the HEALPix column to this VOTable. <br>
**Note:** The _build_healpix_table()_ method sorts the table and removes the duplicated lines in the healpix table automatically to avoid overlapping problems in the future. 
To avoid this, one can specify whether to do it or not with the argument *_rm_duplicates*.


In [149]:
healpix_table = loader.build_healpix_table(votable)
print(healpix_table)

Removed 4344 duplicates.

recno JP11     LID     n_LID   U    ...   H    ref     _RA       _DE    HEALPix
                              mag   ...  mag           deg       deg           
----- ---- ----------- ----- ------ ... ------ ---- --------- --------- -------
  737  737  1.00018884           -- ...     -- 8002  45.56988   4.08973       0
 5938 5938  9.80077031           -- ...  8.100 8130  48.34167   4.77167       1
  731  731  1.00018604        4.130 ...     -- 8029  44.92876   8.90736       3
  769  769  1.00020665        7.350 ...     -- 8083  49.98341   8.69977       4
  782  782  1.00021364        3.330 ...     -- 8029  51.79230   9.73268       5
 4950 4950  5.20250158        8.780 ...     -- 8039  49.01302  11.62845       7
  669  669  1.00016160        7.580 ...     -- 8029  39.02039   6.88687       8
  694  694  1.00017094        4.650 ...     -- 8029  41.23559  10.11415       9
  655  655  1.00015318        4.120 ...     -- 8029  37.03976   8.46005      10
  686  686  1.

/home/yhenin/miniconda3/envs/healpix/lib/python3.11/site-packages/astropy_healpix/core.py:415: RuntimeWarning: invalid value encountered in lonlat_to_healpix_nested
  healpix_index, dx, dy = func(lon, lat, nside)


In [150]:
print(loader.build_healpix_table(votable[:5], _rm_duplicates=False))
print('______________________________________________________________________________________________________________________')
print(healpix_table[:3])

Duplicates were not removed.

recno JP11     LID     n_LID   U    ...   H    ref     _RA       _DE    HEALPix
                              mag   ...  mag           deg       deg           
----- ---- ----------- ----- ------ ... ------ ---- --------- --------- -------
    1    1  0.00000444           -- ...  6.640 8130  40.17863   1.19868    1111
    2    1  0.00000444           -- ...  6.640 8130  40.17863   1.19868    1111
    3    3  0.00004030       15.340 ...  4.100 8057 282.60583   0.79167    1894
    4    4  0.00004064       13.570 ...  2.390 8057 284.38983   0.45552    1894
    5    5  0.00101522           -- ...  6.280 8140 102.08778   1.21897    1388
______________________________________________________________________________________________________________________
recno JP11     LID     n_LID   U    ...   H    ref     _RA       _DE    HEALPix
                              mag   ...  mag           deg       deg           
----- ---- ----------- ----- ------ ... ------ ----

Some lines seem indeed to be duplicated in the new table.

Now that we have our Table well-prepared, one can continue by creating a MOC tree. This is done by the Moc_tree class. <br>

This class has 4 methods:
- _build_moc_tree()_ : create the tree with from a HEALPix table
- _serialize_moc()_ : return the serialized moc tree
- _add_node()_ : add a node to a given order in the tree
- _del_nodes()_ : remove different nodes from a given order in the tree

Let's now build our Moc_tree.

In [151]:
moc_tree = Moc_tree(ORDER)
print(moc_tree.nodes)

{4: []}


There is nothing but the initial order in the tree nodes yet.

In [152]:
# Automatically raises an error if the wrong table is given
# moc_tree.build_moc_tree(votable)

In [153]:
moc_tree.build_moc_tree(healpix_table)
print(moc_tree.nodes)

{4: [0, 1, 3, 4, 5, 7, 13, 15, 18, 19, 25, 26, 34, 35, 41, 42, 43, 45, 46, 47, 52, 55, 56, 57, 60, 62, 63, 68, 69, 71, 73, 74, 75, 76, 79, 80, 83, 84, 85, 87, 94, 95, 96, 97, 99, 101, 102, 106, 107, 117, 118, 120, 122, 124, 126, 130, 131, 132, 134, 136, 137, 138, 142, 144, 146, 147, 148, 150, 153, 154, 155, 157, 166, 168, 170, 171, 173, 176, 177, 178, 196, 198, 199, 204, 205, 206, 214, 216, 218, 219, 232, 236, 242, 243, 252, 254, 255, 261, 264, 266, 268, 269, 271, 272, 276, 277, 282, 284, 285, 286, 288, 289, 298, 304, 305, 312, 313, 314, 319, 322, 326, 327, 331, 332, 334, 335, 336, 338, 339, 340, 341, 343, 345, 346, 347, 348, 349, 351, 352, 354, 357, 360, 361, 362, 364, 365, 379, 380, 382, 384, 385, 387, 388, 389, 390, 393, 396, 397, 399, 404, 406, 415, 417, 419, 420, 423, 425, 430, 431, 432, 433, 435, 443, 444, 445, 446, 448, 450, 451, 462, 463, 466, 467, 469, 470, 471, 472, 474, 475, 480, 485, 491, 496, 501, 503, 506, 508, 512, 517, 524, 526, 528, 529, 531, 533, 534, 535, 536, 537, 5

The tree is built. Let's have a look at the serialization of the tree

In [154]:
print(moc_tree.serialize_moc())

3/2 5 7 12 16 22 27 28 40 45 46 47 48 50 52 56 57 64 73 77 113 135 151 153 168 170 186 202 203 210 214 219 221 222 223 224 225 229 231 232 240 243 244 245 279 295 301 325 327 329 332 335 336 338 344 351 353 356 357 358 365 366 367 369 370 372 377 378 382 394 432 445 447 452 453 454 461 472 474 477 517 520 528 548 581 582 585 588 592 610 612 614 615 617 620 623 637 655 660 662 663 666 672 677 695 723 758 
4/0 1 3 4 5 7 13 15 18 19 25 26 34 35 41 42 43 45 46 47 52 55 56 57 60 62 63 68 69 71 73 74 75 76 79 80 83 84 85 87 94 95 96 97 99 101 102 106 107 117 118 120 122 124 126 130 131 132 134 136 137 138 142 144 146 147 148 150 153 154 155 157 166 168 170 171 173 176 177 178 196 198 199 204 205 206 214 216 218 219 232 236 242 243 252 254 255 261 264 266 268 269 271 272 276 277 282 284 285 286 288 289 298 304 305 312 313 314 319 322 326 327 331 332 334 335 336 338 339 340 341 343 345 346 347 348 349 351 352 354 357 360 361 362 364 365 379 380 382 384 385 387 388 389 390 393 396 397 399 404 4

### Comparison of the results
Now that we have serialized our moc tree, let's compare the results of our custom MOC library with the official mocpy library

In [155]:
from mocpy import MOC

# A MOC object can be obtained from our serialization using MOC.from_string()
custom_moc = MOC.from_string(moc_tree.serialize_moc())

print(custom_moc)

3/2 5 7 12 16 22 27-28 40 45-48 50 52 56-57 64 73 77 113 135 151 153 168 170 
 186 202-203 210 214 219 221-225 229 231-232 240 243-245 279 295 301 325 327 
 329 332 335-336 338 344 351 353 356-358 365-367 369-370 372 377-378 382 394 
 432 445 447 452-454 461 472 474 477 517 520 528 548 581-582 585 588 592 610 
 612 614-615 617 620 623 637 655 660 662-663 666 672 677 695 723 758 
4/0-1 3-5 7 13 15 18-19 25-26 34-35 41-43 45-47 52 55-57 60 62-63 68-69 71 
 73-76 79-80 83-85 87 94-97 99 101-102 106-107 117-118 120 122 124 126 130-132 
 134 136-138 142 144 146-148 150 153-155 157 166 168 170-171 173 176-178 196 
 198-199 204-206 214 216 218-219 232 236 242-243 252 254-255 261 264 266 
 268-269 271-272 276-277 282 284-286 288-289 298 304-305 312-314 319 322 
 326-327 331-332 334-336 338-341 343 345-349 351-352 354 357 360-362 364-365 
 379-380 382 384-385 387-390 393 396-397 399 404 406 415 417 419-420 423 425 
 430-433 435 443-446 448 450-451 462-463 466-467 469-472 474-475 480 485 491 
 4

In [156]:
# One can query the table using the mocpy library as well
moc = MOC.from_vizier_table(table_id=CATALOGUE, nside=2**ORDER)
print(moc)

3/2 5 7 12 16 22 27-28 40 45-48 50 52 56-57 64 73 77 113 135 151 153 168 170 
 186 202-203 210 214 219 221-225 229 231-232 243-245 279 295 301 325 327 329 
 332 335-336 338 344 351 353 356-358 365-367 369-370 372 377-378 382 394 432 
 445 447 452-454 461 472 474 477 517 520 528 548 581-582 585 588 592 610 612 
 614-615 617 620 623 637 655 660 662-663 666 672 677 695 723 758 
4/0-1 3-5 7 13 15 18-19 25-26 34-35 41-43 45-47 52 55-57 60 62-63 68-69 71 
 73-76 79-80 83-85 87 94-97 99 101-102 106-107 117-118 120 122 124 126 130-132 
 134 136-138 142 144 146-148 150 153-155 157 166 168 170-171 173 176-178 196 
 198-199 204-206 214 216 218-219 232 236 242-243 252 254-255 261 264 266 
 268-269 271-272 276-277 282 284-286 288-289 298 304-305 312-314 319 322 
 326-327 331-332 334-336 338-341 343 345-349 351-352 354 357 360-362 364-365 
 379-380 382 384-385 387-390 393 396-397 399 404 406 415 417 419-420 423 425 
 430-433 435 443-446 448 450-451 462-463 466-467 469-472 474-475 480 485 491 
 496 5

Let's check if there are any difference between my serialization and the one that comes from MOCpy. 
I am defining the following function that takes in argument two strings and returns the different elements between the two strings

In [157]:
def difference(string1: str, string2: str):
    # Split both strings into list items
    string1 = string1.split()
    string2 = string2.split()

    set1 = set(string1)
    set2 = set(string2)

    str_diff = set1.symmetric_difference(set2)

    if len(str_diff) == 0:
        print("The MOC strings are the same")
    else:
        print(f"Difference between two MOC strings:\n{str_diff}")


difference(custom_moc.to_string(), moc.to_string())

Difference between two MOC strings:
{'2239-2240', '2239', '240', '960-962', '526'}


Nice, now that we have everything, seeing how they appear on a map would be interesting to verify our work.
<end>

In [158]:
import matplotlib.pyplot as plt
%matplotlib notebook

# Create a figure and axis
fig = plt.figure(figsize=(10, 10))
wcs = custom_moc.wcs(fig)
ax = fig.add_subplot(projection=wcs)

# # Plot the custom MOC in red and the standard MOC in blue
custom_moc.fill(ax, wcs, color='blue', alpha=0.5, label='Custom MOC')
moc.fill(ax, wcs, color='red', alpha=0.5, label='MOCpy')

plt.title("Projection of the HEALPix cells")
plt.legend()
plt.show()  

<IPython.core.display.Javascript object>

/home/yhenin/miniconda3/envs/healpix/lib/python3.11/site-packages/matplotlib/backends/backend_nbagg.py:181: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm('matplotlib', data={'id': self.uuid})


One can see some outliers unfortunately but the major cells are overlapping correctly